In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
from load_data_gpu import UrbanSoundDataset
import torchaudio
from cnn import CNNNetwork

There are 873 samples in the dataset.
tensor([[[1.4664e-03, 2.7458e-04, 1.9349e-03,  ..., 2.8299e-03,
          1.3179e-03, 6.2801e-04],
         [3.8170e-04, 2.9188e-03, 1.4405e-02,  ..., 4.8504e-02,
          1.0698e-02, 4.3954e-03],
         [2.0566e-03, 3.4303e-02, 4.1252e-02,  ..., 9.0840e-02,
          3.2409e-02, 1.4535e-02],
         ...,
         [3.1938e-06, 2.4180e-06, 2.5583e-06,  ..., 1.6207e-04,
          1.1834e-04, 3.6929e-05],
         [2.7774e-06, 1.6969e-06, 1.3460e-06,  ..., 1.1454e-04,
          1.9588e-04, 6.4826e-05],
         [3.1109e-06, 2.1604e-06, 2.2464e-06,  ..., 6.4307e-05,
          8.7382e-05, 4.4857e-05]]], device='cuda:0')
3
torch.Size([1, 64, 44])
cuda:0



[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: C:\Users\Jay\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 66, 46]             160
              ReLU-2           [-1, 16, 66, 46]               0
         MaxPool2d-3           [-1, 16, 33, 23]               0
            Conv2d-4           [-1, 32, 35, 25]           4,640
              ReLU-5           [-1, 32, 35, 25]               0
         MaxPool2d-6           [-1, 32, 17, 12]               0
            Conv2d-7           [-1, 64, 19, 14]          18,496
              ReLU-8           [-1, 64, 19, 14]               0
         MaxPool2d-9             [-1, 64, 9, 7]               0
           Conv2d-10           [-1, 128, 11, 9]          73,856
             ReLU-11           [-1, 128, 11, 9]               0
        MaxPool2d-12            [-1, 128, 5, 4]               0
          Flatten-13                 [-1, 2560]               0
           Linear-14                   

In [2]:
# Steup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
def create_data_loader(train_data, batch_size):
    train_dataloader = DataLoader(train_data, batch_size=batch_size)
    return train_dataloader

In [4]:
ANNOTATIONS_FILE = r"E:\DL_audio\with_pytorch\data1\metadata\fold_1_annotation.csv"
AUDIO_DIR = r"E:\DL_audio\with_pytorch\data1\audio"

SAMPLE_RATE=22050
NUM_SAMPLES = 22050

mel_spec = torchaudio.transforms.MelSpectrogram(sample_rate = SAMPLE_RATE,
                                              n_fft = 1024,
                                              hop_length = 512,
                                              n_mels = 64)
    
usd = UrbanSoundDataset(ANNOTATIONS_FILE,AUDIO_DIR,mel_spec,SAMPLE_RATE,NUM_SAMPLES,device)

signal , label = usd[0]

print(signal) , print(label) , print(signal.shape), print(signal.device)

tensor([[[1.7433e-03, 1.6207e-04, 4.0781e-04,  ..., 2.0526e-03,
          6.8678e-03, 1.7846e-02],
         [6.4027e-04, 5.4754e-03, 2.8272e-02,  ..., 5.5625e-03,
          8.6759e-02, 5.2350e-02],
         [1.0592e-03, 1.2941e-02, 8.1117e-02,  ..., 5.6558e-03,
          6.1651e-02, 1.0182e-01],
         ...,
         [4.8316e-06, 2.0553e-06, 2.5774e-06,  ..., 3.5377e-06,
          3.0520e-06, 4.1328e-06],
         [4.0104e-06, 4.2231e-06, 4.8558e-06,  ..., 1.7981e-06,
          3.2201e-06, 6.0627e-06],
         [4.0844e-06, 4.0532e-06, 4.6760e-06,  ..., 1.6818e-06,
          3.3693e-06, 2.2788e-06]]], device='cuda:0')
3
torch.Size([1, 64, 44])
cuda:0


(None, None, None, None)

train = 0.8*len(usd)

train_data = usd[:train]
test_data = usd[train:]

In [5]:
# Create a dataloader
Batch=128

train_data_loader = create_data_loader(usd, batch_size=Batch)
#test_data_loader = create_data_loader(test_data, batch_size=Batch)

In [7]:
loaded_model_0 = CNNNetwork().to(device)
loaded_model_0.load_state_dict(torch.load(f="cnnnet.pth"))
print(loaded_model_0)

CNNNetwork(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv4): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear): Linear(in_features=2560, out_features=10, bias=True)
  (softmax): Softmax(dim=1)
)


In [8]:
def accuracy_fn(y_true,y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred))*100
    return acc

In [9]:
def train_step(model, data_loader, loss_fn, accuracy_fn, optimizer, device):
    
    model.train()
    for X,y in data_loader:
        X , y = X.to(device) , y.to(device)
        
        y_logits = model(X).squeeze()
        y_preds = torch.argmax(torch.round(torch.sigmoid(y_logits)),dim=1)
        
        #print(y_preds) , print(y)
        
        loss = loss_fn(y_logits,y)
        acc = accuracy_fn(y,y_preds)
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
        
    print(f"Loss: {loss}| Acc: {acc}")
    

def test_step(model, data_loader, loss_fn, device):
    
    model.eval()
    
    with torch.inference_mode():
        
        for X,y in data_loader:
            X , y = X.to(device) , y.to(device)

            test_pred = model(X)

            test_loss = loss_fn(test_pred,y)

        
    print(f"Test Loss: {test_loss}")

In [11]:
LEARNING_RATE = 0.001

loss_fn = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(params=loaded_model_0.parameters(),
                            lr=LEARNING_RATE,
                           weight_decay=0.0001)


In [12]:
# Set the number of epochs
epochs = 500

for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch} \n-----------")
    ### Training
    train_step(model=loaded_model_0,
                   data_loader=train_data_loader,
                   loss_fn=loss_fn,
                   optimizer=optimizer,
                   accuracy_fn=accuracy_fn,
                   device=device)
    
      

#       ###testing
#       test_step(model=model_0,
#                    data_loader=test_data_loader,
#                    loss_fn=loss_fn,
#                    device=device)

  0%|          | 0/500 [00:00<?, ?it/s]

Epoch: 0 
-----------
Loss: 1.873665452003479| Acc: 41.904761904761905
Epoch: 1 
-----------
Loss: 1.8747897148132324| Acc: 41.904761904761905
Epoch: 2 
-----------
Loss: 1.8646284341812134| Acc: 43.80952380952381
Epoch: 3 
-----------
Loss: 1.8643865585327148| Acc: 42.857142857142854
Epoch: 4 
-----------
Loss: 1.8724570274353027| Acc: 40.95238095238095
Epoch: 5 
-----------
Loss: 1.9007394313812256| Acc: 31.428571428571427
Epoch: 6 
-----------
Loss: 1.865067958831787| Acc: 36.19047619047619
Epoch: 7 
-----------
Loss: 1.846276879310608| Acc: 40.95238095238095
Epoch: 8 
-----------
Loss: 1.8491833209991455| Acc: 37.142857142857146
Epoch: 9 
-----------
Loss: 1.8456891775131226| Acc: 38.095238095238095
Epoch: 10 
-----------
Loss: 1.8534879684448242| Acc: 36.19047619047619
Epoch: 11 
-----------
Loss: 1.8473864793777466| Acc: 41.904761904761905
Epoch: 12 
-----------
Loss: 1.8478716611862183| Acc: 41.904761904761905
Epoch: 13 
-----------
Loss: 1.8466911315917969| Acc: 40.952380952380

Loss: 1.8389359712600708| Acc: 41.904761904761905
Epoch: 116 
-----------
Loss: 1.8387727737426758| Acc: 40.0
Epoch: 117 
-----------
Loss: 1.8384846448898315| Acc: 39.04761904761905
Epoch: 118 
-----------
Loss: 1.838424563407898| Acc: 39.04761904761905
Epoch: 119 
-----------
Loss: 1.838716745376587| Acc: 38.095238095238095
Epoch: 120 
-----------
Loss: 1.838187336921692| Acc: 38.095238095238095
Epoch: 121 
-----------
Loss: 1.8386914730072021| Acc: 37.142857142857146
Epoch: 122 
-----------
Loss: 1.8379749059677124| Acc: 38.095238095238095
Epoch: 123 
-----------
Loss: 1.8386225700378418| Acc: 36.19047619047619
Epoch: 124 
-----------
Loss: 1.837801456451416| Acc: 39.04761904761905
Epoch: 125 
-----------
Loss: 1.8383196592330933| Acc: 36.19047619047619
Epoch: 126 
-----------
Loss: 1.8376973867416382| Acc: 39.04761904761905
Epoch: 127 
-----------
Loss: 1.8382422924041748| Acc: 36.19047619047619
Epoch: 128 
-----------
Loss: 1.837608814239502| Acc: 39.04761904761905
Epoch: 129 
---

Loss: 1.840505838394165| Acc: 45.714285714285715
Epoch: 230 
-----------
Loss: 1.8768681287765503| Acc: 40.95238095238095
Epoch: 231 
-----------
Loss: 1.8368860483169556| Acc: 44.761904761904766
Epoch: 232 
-----------
Loss: 1.8556432723999023| Acc: 40.95238095238095
Epoch: 233 
-----------
Loss: 1.8442920446395874| Acc: 43.80952380952381
Epoch: 234 
-----------
Loss: 1.8574211597442627| Acc: 40.95238095238095
Epoch: 235 
-----------
Loss: 1.8478026390075684| Acc: 40.95238095238095
Epoch: 236 
-----------
Loss: 1.8461371660232544| Acc: 42.857142857142854
Epoch: 237 
-----------
Loss: 1.8407279253005981| Acc: 41.904761904761905
Epoch: 238 
-----------
Loss: 1.8507875204086304| Acc: 41.904761904761905
Epoch: 239 
-----------
Loss: 1.8483511209487915| Acc: 41.904761904761905
Epoch: 240 
-----------
Loss: 1.8368901014328003| Acc: 42.857142857142854
Epoch: 241 
-----------
Loss: 1.835658311843872| Acc: 42.857142857142854
Epoch: 242 
-----------
Loss: 1.835358738899231| Acc: 42.857142857142

Loss: 1.842225193977356| Acc: 43.80952380952381
Epoch: 343 
-----------
Loss: 1.828458547592163| Acc: 44.761904761904766
Epoch: 344 
-----------
Loss: 1.8484910726547241| Acc: 42.857142857142854
Epoch: 345 
-----------
Loss: 1.8789640665054321| Acc: 41.904761904761905
Epoch: 346 
-----------
Loss: 1.8339757919311523| Acc: 43.80952380952381
Epoch: 347 
-----------
Loss: 1.8249870538711548| Acc: 46.666666666666664
Epoch: 348 
-----------
Loss: 1.8252041339874268| Acc: 45.714285714285715
Epoch: 349 
-----------
Loss: 1.8251793384552002| Acc: 45.714285714285715
Epoch: 350 
-----------
Loss: 1.825150966644287| Acc: 43.80952380952381
Epoch: 351 
-----------
Loss: 1.824690341949463| Acc: 43.80952380952381
Epoch: 352 
-----------
Loss: 1.8257402181625366| Acc: 42.857142857142854
Epoch: 353 
-----------
Loss: 1.8255411386489868| Acc: 42.857142857142854
Epoch: 354 
-----------
Loss: 1.8255146741867065| Acc: 42.857142857142854
Epoch: 355 
-----------
Loss: 1.8245385885238647| Acc: 43.809523809523

Loss: 1.819365382194519| Acc: 42.857142857142854
Epoch: 455 
-----------
Loss: 1.8192238807678223| Acc: 44.761904761904766
Epoch: 456 
-----------
Loss: 1.8184959888458252| Acc: 43.80952380952381
Epoch: 457 
-----------
Loss: 1.8181788921356201| Acc: 43.80952380952381
Epoch: 458 
-----------
Loss: 1.8174917697906494| Acc: 43.80952380952381
Epoch: 459 
-----------
Loss: 1.817496418952942| Acc: 43.80952380952381
Epoch: 460 
-----------
Loss: 1.8169275522232056| Acc: 43.80952380952381
Epoch: 461 
-----------
Loss: 1.8167731761932373| Acc: 43.80952380952381
Epoch: 462 
-----------
Loss: 1.8165866136550903| Acc: 43.80952380952381
Epoch: 463 
-----------
Loss: 1.817285418510437| Acc: 43.80952380952381
Epoch: 464 
-----------
Loss: 1.8169163465499878| Acc: 43.80952380952381
Epoch: 465 
-----------
Loss: 1.8169183731079102| Acc: 42.857142857142854
Epoch: 466 
-----------
Loss: 1.8161466121673584| Acc: 45.714285714285715
Epoch: 467 
-----------
Loss: 1.8169026374816895| Acc: 42.857142857142854


In [13]:
loaded_model_0.state_dict()

OrderedDict([('conv1.0.weight',
              tensor([[[[ 1.2736e-01, -2.4879e-01, -8.6805e-02],
                        [ 9.9421e-02,  1.7928e-01, -2.1005e-01],
                        [-8.3295e-02, -3.7298e-02, -2.2634e-01]]],
              
              
                      [[[-1.9501e-02, -2.0520e-01,  1.1310e-04],
                        [-3.1207e-01,  1.9089e-01, -2.3956e-01],
                        [ 2.8661e-01,  2.8142e-01, -8.9007e-04]]],
              
              
                      [[[ 2.5945e-01, -2.2633e-01,  1.6707e-01],
                        [ 1.8872e-01, -1.3070e-01, -2.1801e-01],
                        [-2.8751e-01,  3.2580e-01, -1.4293e-01]]],
              
              
                      [[[ 1.6666e-01, -2.6929e-01, -6.4499e-02],
                        [ 3.0797e-01,  2.6599e-01, -2.3795e-01],
                        [ 2.4650e-01,  5.0345e-02, -2.2171e-01]]],
              
              
                      [[[-1.0181e-01, -1.0298e-01, -2.4623e-

In [15]:
torch.save(loaded_model_0.state_dict(), "loaded_cnnnet.pth")
print("Model stored")

Model stored
